In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cos
import scipy.sparse as sp
from scipy.linalg import fractional_matrix_power, inv
import pandas as pd
import random

: 

In [2]:
ass=pd.read_csv('AssociationMatrix.csv',header=None).values
pi_net=pd.read_csv('touying_ncrna.csv',header=None).values
dis_net=pd.read_csv('touying_drug.csv',header=None).values

In [3]:
def construct_adj(adj,pi,dis):
    mat1=np.hstack((pi,adj))
    mat2=np.hstack((adj.T,dis))
    mat=np.vstack((mat1,mat2))
    return mat

In [4]:
adj_raw=construct_adj(ass,pi_net,dis_net)

In [5]:
adj_raw.shape

(746, 746)

In [6]:
pi_feat=pd.read_csv('Final_pi_sim_Feature.csv',header=None)
dis_feat=pd.read_csv('Final_dis_sim_Feature.csv',header=None)


FileNotFoundError: [Errno 2] No such file or directory: 'Final_pi_sim_Feature.csv'

In [15]:
featue=pd.concat([pi_feat,dis_feat],axis=0,ignore_index=True).values

In [9]:
featue=pd.read_csv('node2vec_drug_mi.csv',header=None).values

In [ ]:
#####存为npz

In [10]:
adj_raw=sp.csc_matrix(adj_raw)
sp.save_npz('ori_adj.npz', adj_raw)


In [11]:
featue=sp.csc_matrix(featue)
sp.save_npz('feat.npz', featue)

In [13]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cos
import scipy.sparse as sp
from scipy.linalg import fractional_matrix_power, inv


def knn(feat, num_node, k, data_name, view_name):
    adj = np.zeros((num_node, num_node), dtype=np.int64)
    dist = cos(feat)
    col = np.argpartition(dist, -(k + 1), axis=1)[:, -(k + 1):].flatten()
    adj[np.arange(num_node).repeat(k + 1), col] = 1  
    adj = sp.coo_matrix(adj)
    sp.save_npz(view_name+"_knn.npz", adj)


def adj(adj, data_name, view_name):
    adj = sp.coo_matrix(adj)
    sp.save_npz(view_name+"_adj.npz", adj)


def diff(adj, alpha, data_name, view_name):   
    d = np.diag(np.sum(adj, 1))                                    
    dinv = fractional_matrix_power(d, -0.5)                       
    at = np.matmul(np.matmul(dinv, adj), dinv)                      
    adj = alpha * inv((np.eye(adj.shape[0]) - (1 - alpha) * at))   
    adj = sp.coo_matrix(adj)
    sp.save_npz(view_name+"_diff.npz", adj)

data_name = "ncrna"
view_name = "v2"  # v1 or v2
view_type = "knn"  # knn adj diff

adj = sp.load_npz("ori_adj.npz")####
print(adj)
adj.toarray()
num_node = adj.shape[0]
feat = sp.load_npz("feat.npz")
feat

a = adj.A

# if a[0, 0] == 0:
#     a += np.eye(num_node)
#     print("self-loop!")
adj = a
view_type = "diff" 
alpha=0.1
if view_type == "knn":  # set k
    knn(feat, num_node, 5, data_name, view_name)
elif view_type == "adj":
    adj(adj, data_name, view_name)
elif view_type == "diff":  # set alpha: 0~1
    diff(adj, alpha, data_name, view_name)

  (627, 0)	1.0
  (628, 0)	1.0
  (629, 0)	1.0
  (625, 1)	1.0
  (630, 1)	1.0
  (625, 2)	1.0
  (634, 2)	1.0
  (635, 2)	1.0
  (636, 2)	1.0
  (632, 3)	1.0
  (637, 3)	1.0
  (177, 4)	1.0
  (182, 4)	1.0
  (261, 4)	1.0
  (625, 4)	1.0
  (638, 4)	1.0
  (639, 4)	1.0
  (640, 4)	1.0
  (625, 5)	1.0
  (635, 5)	1.0
  (641, 5)	1.0
  (625, 6)	1.0
  (634, 6)	1.0
  (642, 6)	1.0
  (625, 7)	1.0
  :	:
  (243, 728)	1.0
  (222, 729)	1.0
  (222, 730)	1.0
  (222, 731)	1.0
  (231, 732)	1.0
  (231, 733)	1.0
  (235, 734)	1.0
  (239, 735)	1.0
  (241, 736)	1.0
  (475, 736)	1.0
  (243, 737)	1.0
  (245, 737)	1.0
  (243, 738)	1.0
  (245, 738)	1.0
  (247, 739)	1.0
  (247, 740)	1.0
  (250, 740)	1.0
  (578, 740)	1.0
  (248, 741)	1.0
  (250, 741)	1.0
  (260, 742)	1.0
  (414, 742)	1.0
  (270, 743)	1.0
  (319, 744)	1.0
  (521, 745)	1.0


In [14]:
import torch

def get_khop_indices(k, view):
    view = (view.A > 0).astype("int32")
    view_ = view
    for i in range(1, k):
        view_ = (np.matmul(view_, view.T)>0).astype("int32")
    view_ = torch.tensor(view_).to_sparse()
    return view_.indices()
    
def topk(k, adj):
    pos = np.zeros(adj.shape)
    adj=adj.A
    print(adj)

    for i in range(len(adj)):
      one = adj[i].nonzero()[0]
      if len(one)>k:
        oo = np.argsort(-adj[i, one])
        sele = one[oo[:k]]
        pos[i, sele] = adj[i, sele]
      else:
        pos[i, one] = adj[i, one]
    return pos

#####################
## get k-hop scope ##
## take citeseer   ##
#####################
adj = sp.load_npz("v1_knn.npz")
indice = get_khop_indices(2, adj)
torch.save(indice, "v1_2.pt")


#####################
## get top-k scope ##
## take citeseer   ##
#####################
adj = sp.load_npz("v2_diff.npz")
kn = topk(40, adj)
kn = sp.coo_matrix(kn)
indice = get_khop_indices(1, kn)
torch.save(indice, "v2_40.pt")



[[1.80281372e-01 2.15145022e-04 3.90613094e-04 ... 3.11782137e-04
  2.60461789e-04 1.09814523e-04]
 [2.05599853e-04 1.00535064e-01 4.45352984e-04 ... 1.81407027e-04
  1.73218913e-04 2.74846789e-04]
 [3.81311811e-04 4.52125979e-04 1.26917457e-01 ... 2.50314494e-04
  1.97924141e-04 1.59994660e-04]
 ...
 [3.10597873e-04 1.84326073e-04 2.50340996e-04 ... 1.09049768e-01
  1.30919100e-04 9.65088397e-05]
 [2.51877666e-04 1.74295523e-04 1.96315247e-04 ... 1.30275661e-04
  1.07121062e-01 9.09608494e-05]
 [1.07269720e-04 2.75195580e-04 1.56684795e-04 ... 9.48182846e-05
  8.94182186e-05 1.68294294e-01]]
